In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
from bs4 import BeautifulSoup

DONT RUN ALL

In [2]:
data = pd.read_csv("../data/berry_data_cleaned.csv")
data

,berry,observation_date,main_tree_type,forest_development_class,x,y,year,month,day,week_number,monthday,year-month-day,xy,lat,lon,latlon
0,bilberry,2018-07-07,pine,old,410000,7040000,2018,7,7,27,707,2018-07-07,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-07-07-bil...
1,lingonberry,2018-08-06,pine,old,410000,7040000,2018,8,6,32,806,2018-08-06,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-08-06-lin...
2,cloudberry,2018-07-07,pine,clearing,410000,7040000,2018,7,7,27,707,2018-07-07,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-07-07-clo...
3,bilberry,2022-07-16,pine,old,410000,7040000,2022,7,16,28,716,2022-07-16,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2022-07-16-bil...
4,lingonberry,2022-08-28,pine,old,410000,7040000,2022,8,28,34,828,2022-08-28,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2022-08-28-lin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,lingonberry,2020-08-12,spruce,seedling,500000,7080000,2020,8,12,33,812,2020-08-12,500000-7080000,63.847317,27.000016,63.847316635241-27.000015662314-2020-08-12-lin...
852,cloudberry,2020-07-22,pine,old,500000,7080000,2020,7,22,30,722,2020-07-22,500000-7080000,63.847317,27.000016,63.847316635241-27.000015662314-2020-07-22-clo...
853,cloudberry,2020-07-15,pine,seedling,560000,7140000,2020,7,15,29,715,2020-07-15,560000-7140000,64.380480,28.243717,64.380480243615-28.243717349216-2020-07-15-clo...
854,cloudberry,2020-07-18,pine,clearing,400000,7080000,2020,7,18,29,718,2020-07-18,400000-7080000,63.833034,24.967397,63.833033637422-24.967397132992-2020-07-18-clo...


In [3]:
results = rg.search((63.476942, 25.193479))
results
#region = results[0]['admin1']

Loading formatted geocoded file...


[{'lat': '63.36667',
  'lon': '24.96667',
  'name': 'Kinnula',
  'admin1': 'Central Finland',
  'admin2': 'Saarijaervi-Viitasaari',
  'cc': 'FI'}]

In [3]:
def get_region(row):
    results = rg.search((row['lat'], row['lon']))
    return results[0]['admin1']

In [4]:
data['region'] = data.apply(get_region, axis=1)

Loading formatted geocoded file...


In [5]:
data.to_csv('region_data.csv')

In [6]:
data['region'].value_counts()

Lapland                  161
Northern Ostrobothnia    142
Kainuu                    85
Southern Savonia          70
Central Finland           50
North Karelia             46
Pirkanmaa                 41
Southern Ostrobothnia     39
Northern Savo             35
Varsinais-Suomi           29
Uusimaa                   25
South Karelia             23
Paijanne Tavastia         23
Central Ostrobothnia      21
Haeme                     19
Finnmark Fylke            17
Satakunta                 16
Republic of Karelia        8
Kymenlaakso                4
Ostrobothnia               2
Name: region, dtype: int64

In [7]:
region_data = data['region'].value_counts().to_frame().T

In [8]:
region_codes = {'Lapland': 'FI-10T', 'Northern Ostrobothnia': 'FI-14T', 'Kainuu': 'FI-05T', 'Southern Savonia': 'FI-04T', 'Central Finland': 'FI-08T', 'North Karelia': 'FI-13T', 'Pirkanmaa': 'FI-11T', 'Southern Ostrobothnia': 'FI-03T', 'Northern Savo': 'FI-15T', 'Varsinais-Suomi': 'FI-19T', 'Uusimaa': 'FI-18T', 'South Karelia': 'FI-02T', 'Paijanne Tavastia': 'FI-16T', 'Central Ostrobothnia': 'FI-07T', 'Haeme': 'FI-06T', 'Satakunta': 'FI-17T', 'Kymenlaakso': 'FI-09T', 'Ostrobothnia': 'FI-12T'}

In [9]:
region_data = region_data.append(region_codes, ignore_index=True)
region_data = region_data.dropna(axis=1)

C:\Users\Monit\AppData\Local\Temp\ipykernel_10740\3538106780.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_data = region_data.append(region_codes, ignore_index=True)


In [4]:
region_data = pd.read_csv('region_data.csv')

In [5]:
with open("map_page copy.html") as file:
    soup = BeautifulSoup(file, 'html.parser')

for index, row in region_data.T.iterrows():
    element = soup.find(id=row[1])
    element.string = str(row[0])

with open('map_page copy.html', "w") as file:
    file.write(str(soup))

In [6]:
region_data.to_csv('region_data.csv')